In [ ]:
#!pip show pandas
import pandas as pd
from bs4 import BeautifulSoup
import requests 


#table 1 from wiki 
wiki_url="https://en.wikipedia.org/wiki/List_of_U.S._states_and_territories_by_educational_attainment#List"
table_id="education-by-state"

response=requests.get(wiki_url)
soup=BeautifulSoup(response.text,'html.parser')

####
#table 2 from world population review 
gitCloudGuided='https://github.com/allisgoodrn/guided1/raw/main/'
worldPoplReview='data1.csv'

#table 3 from world population review 
gitCloudGuided1='https://github.com/allisgoodrn/guided1/raw/main/'
worldPoplReviewCost='data2.csv'

#fetching data from table 2 & 3
DFcsv_pop=pd.read_csv(gitCloudGuided + worldPoplReview)
DFcsv_cost=pd.read_csv(gitCloudGuided1 + worldPoplReviewCost)

In [ ]:
#cleaning table 1 from wiki
wikiLink_educ=soup.find('table',attrs={'id':table_id})
DFwiki_educ=pd.read_html(str(wikiLink_educ))
DFwiki_educ

In [ ]:
#cleaning table 1 from wiki

DFwiki_educ=DFwiki_educ[0]

In [ ]:
#cleaning table 1 from wiki

# drop a level from a multi-level column index
DFwiki_educ.columns = DFwiki_educ.columns.droplevel(0)


In [ ]:
#cleaning table 1 from wiki

# drop the unwanted columns
DFwiki_educ= DFwiki_educ.drop(["Population over the age of 25", "Population"], axis=1)
# # rename columns for ease
DFwiki_educ= DFwiki_educ.rename(columns={"State or Territory": "State"})
print(DFwiki_educ.head())

In [ ]:
#cleaning table 1 from wiki

# drop unwanted row 
DFwiki_educ=DFwiki_educ.drop([0,53])
DFwiki_educ

In [ ]:
#cleaning table 1 from wiki

DFwiki_educ.reset_index(drop=True, inplace=True)

In [ ]:
#cleaning table 1 from wiki

DFwiki_educ.columns = ['State', 'Pct1','With a BA or higher','Pct3']
DFwiki_educ

In [ ]:
#cleaning table 1 from wiki

DFwiki_educ=DFwiki_educ.drop(columns=['Pct1', 'Pct3'])

In [ ]:
#cleaning table 1 from wiki

#converting a percentage string to numeric
DFwiki_educ['With a BA or higher'] = DFwiki_educ['With a BA or higher'].str.rstrip("%").astype(float)/100
DFwiki_educ

In [ ]:
#cleaning table 2 population

#changing the column name state to State
DFcsv_cost.rename(columns=
              {'state':'State'},inplace = True)

In [ ]:
##these are the columns shared among the DFs
set(DFcsv_pop.columns) &set(DFcsv_cost)&set(DFwiki_educ)

In [ ]:
#cleaning table 2 population
want_pop=['State','Total','WhiteNonHispanic','BlackNonHispanic','IndianNonHispanic', 'AsianNonHispanic', 'Hispanic']
DFcsv_pop=DFcsv_pop.loc[:,want_pop]

In [ ]:
DFcsv_pop.reset_index(drop=True, inplace=True)

In [ ]:
#cleaning table 3 cost
want_cost=['State','costIndex']
DFcsv_cost=DFcsv_cost.loc[:,want_cost]

In [ ]:
#cleaning table 3 cost 
DFcsv_cost=DFcsv_cost.sort_values(by=["State"], inplace= False, ascending=True)

In [ ]:
#merging all three df 
merged_ready = DFcsv_cost.merge(DFwiki_educ,on='State').merge(DFcsv_pop,on='State')

In [ ]:
#changing the column names 
merged_ready.rename(columns=
              {'Total':'Total_Population',
               'WhiteNonHispanic':'European',
              'BlackNonHispanic': 'African',
               'IndianNonHispanic':'Indigenous',
              'AsianNonHispanic':'Asian',
               'Hispanic':'Latinx',
               'costIndex':'Cost_Index'},inplace = True)

In [ ]:
merged_ready=merged_ready[['State', 'Total_Population', 'With a BA or higher','Cost_Index', 'Indigenous','European','Latinx','African','Asian']]

In [ ]:
#bye anything between parenthesis
merged_ready.columns.str.replace('\(.+\)',"", regex=True)

In [ ]:
#bye anything between parenthesis, bye leading-trailing spaces
merged_ready.columns.str.replace('\(.+\)',"", regex=True).str.strip()

In [ ]:
#bye anything between parenthesis, bye leading-trailing spaces, title case
merged_ready.columns.str.replace('\(.+\)',"", regex=True).\
                          str.strip().\
                          str.title()

In [ ]:
merged_ready.columns=merged_ready.columns.str.replace('\(.+\)',"", regex=True).\
                          str.strip().\
                          str.title()
merged_ready

In [ ]:
merged_ready.columns.str.replace(" ",'',regex=False)

In [ ]:
merged_ready.columns=merged_ready.columns.str.replace(" ",'',regex=False)

In [ ]:
merged_ready=merged_ready[~merged_ready.iloc[:,1:].isna().any(axis=1)].copy()
#
merged_ready

In [ ]:
merged_ready=merged_ready.copy()

In [ ]:
merged_ready.to_csv("deliverable1.csv")